In [11]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from memory_profiler import profile
sns.set()

In [110]:
df = pd.read_csv('data/spotify.csv')

In [15]:
df.drop(columns=['id', 'album', 'album_id', 'artist_ids', 'track_number', 'disc_number'], inplace=True)

In [16]:
df = df[df['year'] > 2000]

In [111]:
addition_df = df[['id', 'name', 'artists', 'release_date', 'year']]
df_corr = df.drop(columns=['id', 'artists', 'release_date', 'year'])
df_corr.corr()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
acousticness,1.000000,-0.263217,-0.089169,-0.750852,-0.208176,0.221956,-0.028028,-0.029654,-0.546639,0.064633,-0.396744,-0.022437,-0.223840,-0.166968
danceability,-0.263217,1.000000,-0.100757,0.204838,0.200842,-0.215589,0.026266,-0.110033,0.249541,-0.048358,0.123746,0.239962,0.005479,0.536713
duration_ms,-0.089169,-0.100757,1.000000,0.060516,-0.033808,0.103621,0.002020,0.028942,0.019791,-0.046849,0.024717,-0.097838,-0.008182,-0.183199
energy,-0.750852,0.204838,0.060516,1.000000,0.102561,-0.177750,0.035780,0.134815,0.779267,-0.056160,0.328939,-0.112616,0.266448,0.326418
explicit,-0.208176,0.200842,-0.033808,0.102561,1.000000,-0.130609,0.005282,0.037288,0.106249,-0.062503,0.152545,0.353872,0.008075,-0.009275
instrumentalness,0.221956,-0.215589,0.103621,-0.177750,-0.130609,1.000000,-0.004619,-0.047941,-0.317562,-0.056731,-0.300625,-0.133966,-0.068656,-0.219188
key,-0.028028,0.026266,0.002020,0.035780,0.005282,-0.004619,1.000000,-0.003368,0.025227,-0.127397,0.001951,0.009648,0.005009,0.025592
liveness,-0.029654,-0.110033,0.028942,0.134815,0.037288,-0.047941,-0.003368,1.000000,0.062695,0.001677,-0.078959,0.122034,0.008586,-0.005781
loudness,-0.546639,0.249541,0.019791,0.779267,0.106249,-0.317562,0.025227,0.062695,1.000000,-0.019250,0.337194,-0.213504,0.217914,0.302520
mode,0.064633,-0.048358,-0.046849,-0.056160,-0.062503,-0.056731,-0.127397,0.001677,-0.019250,1.000000,0.007652,-0.040711,0.002438,0.021592


In [112]:
from sklearn.preprocessing import MinMaxScaler
datatypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
normarization = df.select_dtypes(include=datatypes)
for col in normarization.columns:
    MinMaxScaler(col)

In [113]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10)
features = kmeans.fit_predict(normarization)
df['features'] = features
MinMaxScaler(df['features'])

MinMaxScaler(copy=True,
       feature_range=0         1
1         1
2         1
3         7
4         1
         ..
174384    1
174385    1
174386    4
174387    4
174388    4
Name: features, Length: 174389, dtype: int32)

Old SongRecommendation Class

In [84]:
class SongRecommendation():
    def __init__(self, df):
        '''
        Function: Initialize the dataframe as the dataset for training model

        '''
        self.df = df

    
    def recommend(self, songs, amount=1):
        '''
        Function: Get the input song and number of output songs to get the recommendations
        ------
        Input
            songs (string): Name of the favorite song

            amount (int): Number of desired recommendations
        ------
        Output
            rec (dataframe): A dataframe containing recommended song based on
                            their similarity distance towards the input song (sorted)

        '''
        distance = np.array(0) # Array istance of each song towards the input song
        song = self.df[(self.df.name.str.lower() == songs.lower())].head(1) # Input song
        rec = self.df[self.df.name.str.lower() != songs.lower()] # Other songs
        
        # Traverse the dataframe to calculate the similarity of each song towards the input song
        song_input = song.drop(['artists', 'id', 'name', 'release_date', 'year'], axis=1).to_numpy()
        rec_list = rec.drop(['artists', 'id', 'name', 'release_date', 'year'], axis=1).to_numpy()

        # The algorithm was supposed to mostly use pandas built-in functions
        # But then we changed to the vectorization approach to increase
        # the processing speed while traversing the large dataset.
        distance = np.sum(np.absolute(np.subtract(song_input, rec_list)), axis=1)

        rec['distance'] = distance
        rec = rec.sort_values('distance').reset_index(drop=True) # Sort to get the top <amount> recommended songs
        columns = ['name', 'artists',  'year']
        
        song_input_artist = str(song['artists'].values[0]).strip('[]').strip("''").replace("', '"," & ")

        return song_input_artist, rec[columns][:amount].rename(columns={'name': 'Song', 'artists': 'Artist', 'year': 'Year'})

New SongRecommendation Class

In [114]:
class SongRecommendation():
    def __init__(self, df):
        '''
        Function: Initialize the dataframe as the dataset for training model

        '''
        self.df = df

    def recommend(self, songs, amount=1):
        '''
        Function: Get the input song and number of output songs to get the recommendations
        ------
        Input
            songs (string): Name of the favorite song

            amount (int): Number of desired recommendations
        ------
        Output
            rec (dataframe): A dataframe containing recommended song based on
                            their similarity distance towards the input song (sorted)

        '''
        distance = np.array(0) # Array istance of each song towards the input song
        song = self.df[(self.df.name.str.lower() == songs.lower())].head(1) # Input song
        self.df = self.df[self.df.name.str.lower() != songs.lower()] # Other songs

        # Traverse the dataframe to calculate the similarity of each song towards the input song
        song_input = song.drop(['artists', 'id', 'name', 'release_date', 'year'], axis=1).to_numpy()            

        # The algorithm was supposed to mostly use pandas built-in functions
        # But then we changed to the vectorization approach to increase
        # the processing speed while traversing the large dataset.
        distance = np.sum(np.absolute(np.subtract(song_input, self.df.drop(['artists', 'id', 'name',
                                                                            'release_date', 'year'], axis=1).to_numpy())), axis=1)

        self.df['distance'] = distance
        self.df = self.df.sort_values('distance').reset_index(drop=True) # Sort to get the top <amount> recommended songs
        columns = ['name', 'artists',  'year']

        song_input_artist = str(song['artists'].values[0]).strip('[]').strip("''").replace("', '"," & ")

        return song_input_artist, self.df[columns][:amount].rename(columns={'name': 'Song', 'artists': 'Artist', 'year': 'Year'})

In [115]:
recommendations = SongRecommendation(df)

In [116]:
artist, rec = recommendations.recommend("Look What You Made Me Do", 10)

C:\Users\Bin\Anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [117]:
artist

'Taylor Swift'

In [118]:
rec

,Song,Artist,Year
0,Sleep On The Floor,['The Lumineers'],2016
1,Sleep On The Floor,['The Lumineers'],2016
2,Silhouettes - Original Radio Edit,['Avicii'],2012
3,Brother Love's Travelling Salvation Show,['Neil Diamond'],1969
4,Sexbomb,"['Tom Jones', 'Mousse T.']",1999
5,BLUE,"['Troye Sivan', 'Alex Hope']",2015
6,The Race - Remix,"['Tay-K', '21 Savage', 'Young Nudy']",2017
7,I Want A New Drug,['Huey Lewis & The News'],2006
8,Solo Se Vive una Vez,['Azucar Moreno'],1996
9,Just Friends,['Charlie Parker'],1957


In [119]:
#save model in output directory
joblib.dump(recommendations,'song_recommendation.pkl')

['song_recommendation.pkl']

In [120]:
#open file
file = open('song_recommendation.pkl', "rb")
#load the trained model
trained_model = joblib.load(file)

In [121]:
recommendations.recommend("Lemon Tree", 10)

('Fools Garden',                                                 Song  \
 0                        SideLine Watching (Hold Up)   
 1             The Ones That Didn’t Make It Back Home   
 2                                       Guys Like Me   
 3  Needles In The Camel's Eye - 2004 Digital Rema...   
 4                                         I Get High   
 5                                         El Tarasco   
 6                                 Thank God For Kids   
 7         acting like that (feat. Machine Gun Kelly)   
 8            Stop The World I Wanna Get Off With You   
 9                    Day After Day - Remastered 2010   
 
                                               Artist  Year  
 0                                   ['Lil Uzi Vert']  2016  
 1                                   ['Justin Moore']  2019  
 2                                    ['Eric Church']  2006  
 3                                      ['Brian Eno']  1973  
 4                                    ['Freda